In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [ ]:
from langchain.agents import create_agent

# 각각 도구를 가진 서브 에이전트를 정의

subagent_1 = create_agent(
    model='gpt-5-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-5-nano',
    tools=[square]
)

## Calling subagents

In [4]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## 메인 에이전트 정의

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="당신은 숫자의 제곱근이나 제공을 계산하는 서브 에이전트를 호출하는 친절한 어시스턴트입니다.")

## Test

In [5]:
question = "456의 제곱근은?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='456의 제곱근은?', additional_kwargs={}, response_metadata={}, id='c33f012a-493a-4152-91bb-ef76d3e9f3ba'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 859, 'prompt_tokens': 212, 'total_tokens': 1071, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6takZHOYTkgN39zTY4JVGP9FtWfu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3c2b-d74f-70c1-b353-a703b8cf25c6-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_06M3ttqvyvVGhWiGGaQQtEhW', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 212, 'output_tokens': 859, 